<a href="https://colab.research.google.com/github/vaishnavi2810-code/AI-For-Beginners/blob/main/token_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Empirical Regularities of Language

In this first homework assignment, you will familiarize yourself with some empirical regularities of natural language, Shannon entropy and Zipf's Law.

Read through this Jupyter notebook and fill in the parts marked with `TODO`. When you're ready to submit, print the notebook as a PDF and upload to Gradescope.

## Shannon Entropy

Shannon borrowed the concept of entropy from statistical physics to develop _information theory_, focused on encoding and compressing messages. A few years later, in 1950, he applied information theory to analyze human predictive ability—in other words, the entropy of the human language model. You can read the original article, [Prediction and Entropy of Printed English](https://languagelog.ldc.upenn.edu/myl/Shannon1950.pdf), for more details.

Your first task is to collect data on how predictable different letters are in an English sentence, depending on how much context in a word or sentence you have.

Go to the [Shannon game page](https://www.ccs.neu.edu/home/dasmith/courses/cs6120/shannon/) that we demonstrated in class. We already guessed part of Text 1, so work through Texts 2, 3, and 4.

In [5]:
# TODO: Enter the arrays of numbers of guesses for Texts 2, 3, and 4 here.
text2 = [1,7,1,1,1,1,1,1,9,4,1,1,1,5,1,1,1,1,7,1,4,1,1,1,1,1,1,12,1,1,5,1,11,1,1,1,1,1,3,1,1,1,1,1,1,1,1]
text3 = [3,2,27,14,13,2,4,1,5,15,15,1,2,21,1,1,1,1,2,1,1,1,1,1,1,14,3,18,2,1,1,1,1,2,1,15,1,21,1,1,1,4,10,5,27,15,1,1,3,1,1,1,12,19,1,14,1,18,6,3,1,1,1,1,1,1,1]
text4 = [7,3,6,2,2,10,20,2,1,1,1,1,1,1,1,20,6,15,3,1,1,1,1,1,4,2,9,6,20,1,1,1,3,1,20,22,1,1,1,6,1,16,25,1,21,9,3,2,1,1,1,1,1,3,1,1,1]

[ context0: [1,0,1,0,0,0,1],
 context1: [0,1,1,0,0,0,1]

]

print(len(text2), len(text3), len(text4))
print(len("federal officials began to stockpile ammunition"))
print(len("to draw conclusions from materials often discordant is no easy task"))
print(len("each iteration tends to beget a better copy than the last"))

47 67 57
47
67
57


Rearrange the guess data into a two-dimensional array, relating number of characters of context (0, 1, 2, ...) to number of guesses required.

In other words, you might look in cell (2, 1) and read "2" if the number of times it took one guess to get the right letter with two characters of context was 2.

In [15]:
# TODO: Create array of counts of guesses. Print out the array so we can see it.
text2 = [1,7,1,1,1,1,1,1,9,4,1,1,1,5,1,1,1,1,7,1,4,1,1,1,1,1,1,12,1,1,5,1,11,1,1,1,1,1,3,1,1,1,1,1,1,1,1]
import numpy as np
import pandas as pd

def make_context_guess_matrix(guesses):
    """
    Rearrange guess data into a 2D array.
    Row = context size (0, 1, 2, ...)
    Col = number of guesses (1, 2, 3, ...)
    Entry = how many times that many guesses occurred at that context.
    """
    max_context = len(guesses) - 1
    max_guess = max(guesses)

    # matrix with columns = 1..max_guess
    counts = np.zeros((max_context+1, max_guess), dtype=int)

    for context, guess in enumerate(guesses):
        counts[context, guess-1] += 1   # shift by 1

    return counts

matrix_for_text2 = make_context_guess_matrix(text2)
matrix_for_text3 = make_context_guess_matrix(text3)
matrix_for_text4 = make_context_guess_matrix(text4)
#df = pd.DataFrame(matrix, columns=range(1, matrix.shape[1]+1))
#print(df.to_string())


    1   2   3   4   5   6   7   8   9   10  11  12
0    1   0   0   0   0   0   0   0   0   0   0   0
1    0   0   0   0   0   0   1   0   0   0   0   0
2    1   0   0   0   0   0   0   0   0   0   0   0
3    1   0   0   0   0   0   0   0   0   0   0   0
4    1   0   0   0   0   0   0   0   0   0   0   0
5    1   0   0   0   0   0   0   0   0   0   0   0
6    1   0   0   0   0   0   0   0   0   0   0   0
7    1   0   0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   1   0   0   0
9    0   0   0   1   0   0   0   0   0   0   0   0
10   1   0   0   0   0   0   0   0   0   0   0   0
11   1   0   0   0   0   0   0   0   0   0   0   0
12   1   0   0   0   0   0   0   0   0   0   0   0
13   0   0   0   0   1   0   0   0   0   0   0   0
14   1   0   0   0   0   0   0   0   0   0   0   0
15   1   0   0   0   0   0   0   0   0   0   0   0
16   1   0   0   0   0   0   0   0   0   0   0   0
17   1   0   0   0   0   0   0   0   0   0   0   0
18   0   0   0   0   0   0   1 

Now you can compute Shannon's upper and lower bounds on the entropy of your predictive distribution for English. The upper bound, as a function of the number of context characters $N$, is just the Shannon entropy of the distribution of numbers of guesses. In other words, it's the entropy of the original text as &ldquo;reduced&rdquo; by the human encoder to a sequence of numbers of guesses.

$F_N = -\sum_{i=1}^{27} q_i^N \log_2 q_i^N$

where $q_i^N$ is the number of times you took $i$ guesses with $N$ characters of context, i.e., one of the cells in the table you computed. The number of guesses ranges from 1 to 27 since we restrict ourselves to 26 letters plus space. In computing entropy, we define $0 \log 0 = 0$.

In [ ]:
# TODO: Compute the upper bound for each amount of context N and print it out.

Shannon derived a lower bound on entropy from the guess data as

$\sum_{i=1}^{27} i(q_i^N - q_{i+1}^N) \log_2 i$

In [ ]:
# TODO: Compute thew lower bound for each amount of context N and print it out.

## Zipf's Law

Now let's look at some text data directly to see the skewed distribution of tokens predicted by Zipf's Law. Recall that Zipf's law states that a word's rank (from the most common word at rank 1 on down) to its frequency is approximately a constant, i.e., $r \cdot f = k$. Equivalently, we can divide both sides by the total number of tokens $N$ to get $r \cdot P_r = c$, where $c = k/N$ and $P_r = f/N$ is the _relative frequency_ of word $r$.

We start by downloading a sample of 1000 open-access English books from [Project Gutenberg](https://gutenberg.org/).

In [ ]:
# If your local environment doesn't have the wget command,
# you can comment this out and download it manually.
!wget "http://khoury.northeastern.edu/home/dasmith/pg-sample.json.gz"

The file is compressed with gzip and is in a JSON lines format. Each line is one JSON record, which we parse with the `json` library.

Here we print out the keys in the first record: `id`, `author`, `title`, and `text`.

In [ ]:
import gzip, json
for line in gzip.open("pg-sample.json.gz", mode="rt", encoding="utf-8"):
  rec = json.loads(line)
  print(rec.keys())
  print(rec['author'])
  print(rec['title'])
  print(rec['text'][0:100])
  break

dict_keys(['id', 'author', 'title', 'text'])
Jefferson, Thomas
The Declaration of Independence of the United States of America


This is a retranscription of one of the first Project
Gutenberg Etexts, offically dated December 3


Your task now is to **tokenize** the text in the `text` field of each record into an array of words. Later on in this course, we will discuss learning better tokenizers. For now, you should separate words on whitespace (space, newline, tab) and punctuation. Convert the tokens to lower case, and keep only those tokens that have at least one letter a-z in them. In general, numerals in text tend not to follow Zipf's law but [Benford's law](https://en.wikipedia.org/wiki/Benford%27s_law).

You might use _regular expressions_ (e.g., the `re.split` function) to help with tokenization and filtering.

After you have tokenized, compute $N$, the total number of tokens in the corpus and print it out.

In [ ]:
# TODO: Compute an array of tokens in the corpus
# Compute the total number of tokens N and print it out.

Now, count the frequency each unigram (distinct word) in the corpus and sort them in an array in descending order of frequency. The first item in your array should be the most common word. Print out that word and its frequency

In [ ]:
# TODO: Compute an arrary of unigrams in descending order of frequency.
# Print the most common word and its frequency.

Now, you can look at the Zipf's law relationship between rank and relative frequency (i.e., frequency divided by $N$). Plot the data using a python graphing package such as matplotlib, plotly, or plotnine. This doesn't have to be a fancy graph, so use whatever you're familiar with. Both axes should be on a log scale. If your package doesn't support log scales, you can take the log of the rank and relative frequency yourself before plotting. Recall that since python arrays are zero-indexed, the rank 1 word will be element 0 of your sorted array.

In [ ]:
# TODO: Plot rank vs. relative frequency of unigrams.

Now, take your array of tokens and compute the counts of both the bigrams and trigrams and sort them in descending order of frequency. Print out the most common bigram and trigram.

In [ ]:
# TODO: Compute sorted bigram and trigram statistics.
# Print out the most common bigram and trigram.
# Plot rank vs. relative frequency for bigrams and trigrams.
# You may make separate plots or put them on the same plot and label them.

**TODO**: Finally, write your visual impressions of the fit of the unigram, bigram, and trigram distributions. This doesn't need to be statistically rigorous.